# 📈 BTC Time Series Forecasting - ARIMA

This notebook performs time series forecasting on daily BTC network statistics using `auto_arima` from the `pmdarima` library.

Forecasts are generated for transaction count, total fees, and active addresses.

Results are saved under `results/btc/`.

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pmdarima import auto_arima

# Ensure result directory exists

In [23]:
os.makedirs("../../results/btc", exist_ok=True)

# Load data

In [24]:
df = pd.read_csv("../../data/btc/btc_network_daily.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Forecast horizon (e.g., 14 days ahead)

In [25]:
forecast_horizon = 14

# Forecast each variable

In [26]:
# Remove if last day has suspiciously low activity
if df.iloc[-1]['tx_count'] < 250 or df.iloc[-1]['active_addresses'] < 500:
    df = df.iloc[:-1]

In [27]:
for col in ['tx_count', 'total_fees', 'active_addresses']:
    ts = df[col].dropna()

    # Fit auto ARIMA
    model = auto_arima(ts, seasonal=False, stepwise=True, suppress_warnings=True, error_action='ignore')

    # Forecast future values
    forecast = model.predict(n_periods=forecast_horizon)
    forecast_index = pd.date_range(start=ts.index[-1] + pd.Timedelta(days=1), periods=forecast_horizon, freq='D')
    forecast_series = pd.Series(forecast, index=forecast_index, name='forecast')

    # Save forecast to CSV
    forecast_df = pd.DataFrame({
        'Date': forecast_index,
        f'{col}_forecast': forecast
    })
    forecast_df.to_csv(f"../../results/btc/{col}_forecast.csv", index=False)

    # Plot original + forecast
    plt.figure(figsize=(12, 4))
    plt.plot(ts.index, ts.values, label='Historical')
    plt.plot(forecast_index, forecast, label='Forecast', linestyle='--')
    plt.title(f"{col.replace('_', ' ').title()} Forecast (BTC)")
    plt.xlabel("Date")
    plt.ylabel(col.replace('_', ' ').title())
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"../../results/btc/{col}_forecast_plot.png")
    plt.close()

/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/

In [29]:
print("Forecasts completed and saved to `results/btc/`.")

Forecasts completed and saved to `results/btc/`.
